In [1]:
from os import sys
# Path to workspace
sys.path.insert(0, '/workspace/3d-shapes-embeddings/contrib/sharp_features/')
sys.path.insert(0, '/workspace/3d-shapes-embeddings/workspace/')
sys.path.insert(0, '/workspace/dense-self-supervised-representation-learning-for-3D-shapes/')
sys.path.insert(0, '/workspace/')

In [2]:
from models.meshcnn.utils.mesh_prepare import *
import h5py
import numpy as np

In [3]:
class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self

In [4]:
import torch
from torch.utils.data import Dataset, DataLoader
from models.meshcnn.utils.mesh import Mesh


    
    
def pad(input_arr, target_length, val=0, dim=1):
    shp = input_arr.shape
    npad = [(0, 0) for _ in range(len(shp))]
    npad[dim] = (0, target_length - shp[dim])
    return np.pad(input_arr, pad_width=npad, mode='constant', constant_values=val)


def collate_fn(batch, device='cpu'):
    """Creates mini-batch tensors
    We should build custom collate_fn rather than using default collate_fn
    """
    meta = {}
    keys = batch[0].keys()
    for key in keys:
        meta.update({key: np.array([d[key] for d in batch])})
        
    meta['edge_features'].to(device)
    return meta


class MeshDataset(Dataset):
    """
    Dataset for MeshCNN
    """
    def __init__(self, h5_file: str,
                 ninput_edges: int,
                 opt):
        """
        param h5_file: path to h5 file with preprocessed meshes
        param part: part of dataset, can be train/val/test
        """
        super().__init__()
        self.ninput_edges = ninput_edges
        self.file = h5py.File(h5_file, 'r')
        self.opt = opt
        self.get_mean_std()
        
    
    def get_mean_std(self):
        self.mean = 0
        self.std = 0
        for i in range(self.__len__()):
            m = from_scratch(
                (self.file['vertices'][i].reshape(-1, 3), self.file['faces'][i].reshape(-1, 3)),
                self.opt, False
            )
            features = m['features']
            
            self.mean = self.mean + features.mean(axis=1)
            self.std = self.std + features.std(axis=1)
        
        self.mean = self.mean / self.__len__()
        self.std = self.std / self.__len__()
        
        
    def __getitem__(self, index: int):
        mesh = Mesh(from_scratch(
            (self.file['vertices'][index].reshape(-1, 3), self.file['faces'][index].reshape(-1, 3)),
            self.opt, self.opt.is_train
        ), hold_history=True)
        meta = {'mesh': mesh}
        # get edge features
        edge_features = mesh.extract_features()
        edge_features = pad(edge_features, self.ninput_edges)
        meta['edge_features'] = (edge_features - self.mean[..., None]) / self.std[..., None]
        
        return meta
    
    
    def __len__(self):
        return self.file['faces'].shape[0]


In [5]:
from tqdm import tqdm
stats = []
for i in tqdm(range(len(data))):
    stats.append(data[i]['mesh'].features.shape[1])
    
stats = np.array(stats)

NameError: name 'data' is not defined

In [ ]:
stats.max()

In [ ]:
stats.max()

In [6]:
data = MeshDataset(
    'abc_train.hdf5',
    1000,
    opt = AttrDict({
        'normalize': True,
        'num_aug': 1,
        'scale_verts': True,
        'slide_verts': 0.2,
        'flip_edges': 0.2,
        'is_train': True
    })
)

In [18]:
data[0]['mesh'].edges

array([[ 5,  9],
       [ 8,  9],
       [ 5,  8],
       [ 0,  9],
       [ 0,  8],
       [ 0, 10],
       [ 8, 10],
       [ 2, 10],
       [ 2,  8],
       [ 1,  9],
       [ 0,  1],
       [ 4,  8],
       [ 3,  8],
       [ 3,  4],
       [ 4,  5],
       [ 2,  3],
       [ 3, 12],
       [11, 12],
       [ 3, 11],
       [ 7, 12],
       [ 7, 11],
       [ 3, 13],
       [12, 13],
       [ 7, 13],
       [ 4, 11],
       [ 6, 13],
       [ 6,  7],
       [ 5, 11],
       [ 2, 13]], dtype=int32)

In [7]:
from meshcnn.networks import *
from meshcnn.layers.mesh_pool import *

In [8]:
batch = collate_fn([
    data[0], data[1], data[2]
])
batch.keys()

dict_keys(['mesh', 'edge_features'])

In [9]:
sample = MeshEncoderDecoder(
    pools=[0, 600, 512, 256, 128, 64],
    down_convs=[5, 64, 128, 256, 512, 1024],
    up_convs=[1024, 1024, 512, 256, 128, 64],
    blocks=0
)

In [ ]:
res = sample.encoder((torch.from_numpy(batch['edge_features']).float(), batch['mesh']))
[_.shape if _ is not None else None for _ in res[1]]

In [10]:
sample(torch.from_numpy(batch['edge_features']).float(), batch['mesh']).shape

torch.Size([3, 64, 600])

In [ ]:
encoder = MeshEncoder(
    pools=[0, 500, 250, 100],
    convs=[5, 64, 128, 256]
)
decoder = MeshDecoder(
    unrolls=[500, 1000],
    convs=[256, 128, 64],
    transfer_data=False,
)

In [ ]:
encoded = encoder((torch.from_numpy(batch['edge_features']).float(), batch['mesh']))[0] 

In [ ]:
res = decoder((encoded, batch['mesh']))

In [ ]:
res.shape

In [ ]:
batch['mesh'][0].

In [ ]:
torch.from_numpy(batch['edge_features'])[0]

In [ ]:
pad(torch.from_numpy(data[0]['edge_features']), 1000)

In [ ]:
data[0]['edge_features']